In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from astropy.coordinates import SkyCoord
import os
import sys
sys.path.insert(0, '/home/aew492/lss-dipoles')
import tools
from Secrest_dipole import SecrestDipole
from multipoles import multipole_map
import dipole

In [ ]:
# fitting function
def fit_multipole(map_to_fit, template_maps, Cinv=None, fit_zeros=False, idx=None):
    """
    Fit multipole amplitudes to an input healpix density map.
    
    Parameters
    ----------
    map_to_fit : 1D array-like, length npix
        Input healpix map.
    template_maps : 2D array-like, shape (2 * ell + 1, npix)
        Template healpix maps, one for each order m.
        Must be in order of increasing m! i.e. [-m,m]
    Cinv : array-like, optional
        Inverse covariance matrix. If 1D, taken to be the diagonal terms.
    fit_zeros : bool, optional
        Whether to fit zero-valued pixels in `map_to_fit`. The default is False.
    idx : array-like, optional
        Pixel indices to fit.
    
    Returns
    -------
    bestfit_pars :
        The 2 * ell + 1 best-fit amplitudes corresponding to each template map.
    bestfit_stderr :
        The standard error on the fit.
    
    """
    assert map_to_fit.ndim == 1, "input map must be 1-dimensional"
    assert len(map_to_fit) == template_maps.shape[1], "input map and template maps must have the same NPIX"
    
    NPIX = len(map_to_fit)
    # design matrix
    A = np.column_stack((np.ones(NPIX), template_maps.T))
    # covariances: identity for now
    if Cinv is None:
        Cinv = np.ones(NPIX)
    else:
        assert len(Cinv) == NPIX, "input Cinv and input map must have the same length"

    # indices to fit
    idx_to_fit = np.full(NPIX, True)
    if fit_zeros is False:
        idx_to_fit = idx_to_fit & (map_to_fit!=0.)
    if idx is not None:
        assert len(idx) == NPIX, "input idx and input map must have the same length"
        idx_to_fit = idx_to_fit & idx
    map_to_fit, A, Cinv = map_to_fit[idx_to_fit], A[idx_to_fit], Cinv[idx_to_fit]

    # perform the regression
    bestfit_pars, bestfit_Cinv = tools.lstsq(map_to_fit, A, Cinv)

    # uncertainties on the best-fit pars
    bestfit_stderr = np.sqrt(np.diag(np.linalg.inv(bestfit_Cinv)))

    return bestfit_pars, bestfit_stderr

### inputs

In [ ]:
NSIDE = 64

### load sample

In [ ]:
catwise_kwargs = dict(initial_catfn='catwise_agns_master.fits', catname='catwise_agns', mag='w1',
                      blim=30, maglim=16.4, load_init=False)
quaia_kwargs = dict(initial_catfn='quaia_G20.0.fits', catname='quaia', mag='G',
                    blim=30, maglim=20., save_tag='_r1.0', load_init=False, compcorrect=True)
d = SecrestDipole(**catwise_kwargs)
map_ = d.load_hpxelatcorr()

In [ ]:
# construct map from source density table
map_to_fit = np.empty(hp.nside2npix(NSIDE))
map_to_fit[:] = np.nan
map_to_fit[map_['hpidx']] = map_['elatdenscorr']
mean, std = np.nanmean(map_to_fit), np.nanstd(map_to_fit)
fig = plt.figure(figsize=(8,4))
hp.mollview(map_to_fit, coord=['C','G'], title=f'Input map: {d.catname}', unit='sources per healpixel',
            badcolor='w', min=mean-2*std, max=mean+2*std, fig=fig)
hp.graticule()

### dipole
Check that old and new template maps & fitting functions give identical results!

In [ ]:
# maps as constructed in my dipole fitting functions
template_amps = np.column_stack((np.zeros(3), np.diag(np.ones(3)))) # first column for the monopole 
                                                                    #  since dipole_map() takes 4 input amplitudes
old_dipole_templates = np.array([
    dipole.dipole_map(amps, NSIDE=NSIDE) for amps in template_amps
])

fig = plt.figure(figsize=(10,2.5))
titles = ['x', 'y', 'z']
for i, template in enumerate(old_dipole_templates):
    hp.mollview(template, sub=(1,3,i+1), title=titles[i], min=-1, max=1, cmap='coolwarm', fig=fig)
fig.suptitle('Dipole templates (old function)')

In [ ]:
# my new template map construction
ampss = np.identity(3)
new_dipole_templates = np.array([
    multipole_map(amps, NSIDE=NSIDE) for amps in ampss
])  # multipole_map() assumes ell from len(amps) and assumes that amps are given in order of increasing m

fig = plt.figure(figsize=(10,2.5))
titles = ['m = -1', 'm = 0', 'm = 1']
for i, template in enumerate(new_dipole_templates):
    hp.mollview(template, sub=(1,3,i+1), title=titles[i], min=-1, max=1, cmap='coolwarm')
fig.suptitle('Dipole templates (new function)')

In [ ]:
# old fitting function: this automatically fits the monopole + 3 dipole templates
pars, stderr = dipole.fit_dipole(map_to_fit, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# monopole + dipole
bestfit_dipmap = dipole.dipole_map(pars, NSIDE=NSIDE)
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_dipmap, coord=['C','G'], title='Recovered dipole', fig=fig)

In [ ]:
# new fitting function: automatically adds the monopole to the design matrix
pars, stderr = fit_multipole(map_to_fit, new_dipole_templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# monopole + dipole
bestfit_dipmap = multipole_map(pars[0], NSIDE=NSIDE) + multipole_map(pars[1:], NSIDE=NSIDE)
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_dipmap, coord=['C','G'], title='Recovered dipole', fig=fig)

In [ ]:
amp = np.linalg.norm(pars[1:]/pars[0])
# manually tell healpy which parameters correspond to the x, y, and z directions
direction = hp.vec2dir(pars[3], vy=pars[1], vz=pars[2])
direction = SkyCoord(direction[1], np.pi/2 - direction[0], frame='icrs', unit='rad')
amp, direction.galactic

Correct fiducial result!

### quadrupole

In [ ]:
ampss = np.identity(5)
quadrupole_templates = np.array([
    multipole_map(amps, NSIDE=NSIDE) for amps in ampss
])

fig = plt.figure(figsize=(12,2))
titles = ['m = -2', 'm = -1', 'm = 0', 'm = 1', 'm = 2']
for i, template in enumerate(quadrupole_templates):
    hp.mollview(template, sub=(1,5,i+1), title=titles[i], min=-1, max=1, cmap='coolwarm', fig=fig)
fig.suptitle('Quadrupole templates')

In [ ]:
# fit to same map as above
pars, stderr = fit_multipole(map_to_fit, quadrupole_templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# quadrupole scaled by the monopole
bestfit_map = multipole_map(pars[1:], NSIDE=NSIDE) / pars[0]
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_map, coord=['C','G'], title='Recovered dimensionless quadrupole', fig=fig)

In [ ]:
amp = np.linalg.norm(pars[1:]/pars[0])
amp

### octupole

In [ ]:
ampss = np.identity(7)
octupole_templates = np.array([
    multipole_map(amps, NSIDE=NSIDE) for amps in ampss
])

fig = plt.figure(figsize=(12,1.5))
titles = ['m = -3', 'm = -2', 'm = -1', 'm = 0', 'm = 1', 'm = 2', 'm = 3']
for i, template in enumerate(octupole_templates):
    hp.mollview(template, sub=(1,len(ampss),i+1), title=titles[i], min=-1, max=1, cmap='coolwarm', fig=fig)
fig.suptitle('Octupole templates')

In [ ]:
pars, stderr = fit_multipole(map_to_fit, octupole_templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# octupole scaled by the monopole
bestfit_map = multipole_map(pars[1:], NSIDE=NSIDE) / pars[0]
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_map, coord=['C','G'], title='Recovered dimensionless octupole', fig=fig)

In [ ]:
amp = np.linalg.norm(pars[1:]/pars[0])
amp

### dipole + quadrupole

In [ ]:
templates = np.concatenate([new_dipole_templates, quadrupole_templates])

fig = plt.figure(figsize=(15,1.5))
for i, template in enumerate(templates):
    hp.mollview(template, sub=(1,len(templates),i+1), title='', min=-1, max=1, cmap='coolwarm', fig=fig)
fig.suptitle('Dipole + quadrupole templates')

In [ ]:
# fit to same map as above
pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# dipole + quadrupole, scaled by the monopole
bestfit_map = (multipole_map(pars[1:4], NSIDE=NSIDE) + multipole_map(pars[4:])) / pars[0]
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_map, coord=['C','G'], title='Recovered dipole + quadrupole', fig=fig)

In [ ]:
# amplitudes
np.linalg.norm(pars[1:4])/pars[0], np.linalg.norm(pars[4:])/pars[0]

### dipole + quadrupole + octupole

In [ ]:
templates = np.concatenate([new_dipole_templates, quadrupole_templates, octupole_templates])

fig = plt.figure(figsize=(14,2.5))
for i, template in enumerate(np.concatenate([np.ones_like(templates[0])[None,:], templates])):
    hp.mollview(template, sub=(2,round(len(templates)/2),i+1), coord=['C','G'], title='', min=-.6, max=.6, cmap='coolwarm', fig=fig)
fig.suptitle('Dipole + quadrupole + octupole templates')

In [ ]:
# fit to same map as above
pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))
print("best-fit pars: ", pars)

# dipole + quadrupole + octupole, scaled by the monopole
bestfit_map = (multipole_map(pars[1:4], NSIDE=NSIDE) + multipole_map(pars[4:9]) + multipole_map(pars[9:])) / pars[0]
fig = plt.figure(figsize=(7,3))
hp.mollview(bestfit_map, coord=['C','G'], title='Recovered dipole + quadrupole + octupole', fig=fig)

In [ ]:
pars.shape, templates.shape

In [ ]:
hp.mollview((templates.T @ pars[1:]) / pars[0], coord=['C','G'])

In [ ]:
# amplitudes
np.linalg.norm(pars[1:4])/pars[0], np.linalg.norm(pars[4:9])/pars[0], np.linalg.norm(pars[9:])/pars[0]

### $\hat{C}_\ell$

Remember that any well-behaved function of $\theta$ and $\phi$ can be expressed entirely in terms of spherical harmonics (completeness property):
$$
f(\theta,\phi) = \sum_{\ell=0}^{\infty}\sum_{m=-\ell}^{\ell} a_{\ell m}\,Y_\ell^m
$$

Define our estimate as
$$
\hat{C}_\ell = \frac{1}{2\ell +1}\,\sum_{m=-\ell}^{\ell} | a_{\ell m} |^2
$$

In [ ]:
templates[0]

In [ ]:
def compute_Cells(amps):
    ell = 1
    i1 = 0
    Cells = np.array([])
    while i1 < len(amps):
        i2 = i1 + 2 * ell + 1
        assert i2 <= len(amps)
        print(i1, i2, len(amps))
        Cell = compute_Cell(amps[i1:i2])
        Cells = np.append(Cells, Cell)
        ell += 1
        i1 = i2
    return Cells

In [ ]:
def compute_Cell(aellems):
    """
    Return the power C(ell) given a list of coefficients a_ellem.
    """
    assert aellems.ndim <= 1
    # pad if aellems is a scalar:
    if aellems.ndim == 0:
        aellems = aellems[..., np.newaxis]
    # infer ell from the number of moments 2ell+1
    ell = (len(aellems) - 1) // 2
    return np.mean(aellems**2)

In [ ]:
# compute full fit of all the amplitudes
ells = np.arange(1, 8)
templatess = None
for i, ell in enumerate(ells):
    # construct templates for fit
    ampss = np.identity(2 * ell + 1)
    templates = np.array([
        multipole_map(amps, NSIDE=NSIDE) for amps in ampss
    ])  # multipole_map() assumes ell from len(amps) and assumes that amps are given in order of increasing m
    if templatess is None:
        templatess = templates
    else:
        templatess = np.concatenate([templatess, templates])
    
# get the best-fit multipole moments
pars, stderr = fit_multipole(map_to_fit, templatess, idx=~np.isnan(map_to_fit))

In [ ]:
# compute Cell 
Cells = compute_Cells(pars[1:]/pars[0])

In [ ]:
Cells

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(ells, Cells, 'ks')
ax.grid(lw=0.5, alpha=0.5)
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$\hat{C}_\ell$')
ax.set_title(f'{d.catname}'r' low-$\ell$ power spectrum')

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(ells, ells * (ells + 1) * Cells, 'ks')
ax.grid(lw=0.5, alpha=0.5)
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$\ell\,(\ell +1)\,\hat{C}_\ell$')
ax.set_title(f'{d.catname}'r' low-$\ell$ power spectrum')

In [ ]:
# do nothing below this line
assert False

In [ ]:
# other sample
d = SecrestDipole(**quaia_kwargs)
map_ = d.load_hpxelatcorr()
map_to_fit = np.empty(hp.nside2npix(NSIDE))
map_to_fit[:] = np.nan
map_to_fit[map_['hpidx']] = map_['elatdenscorr']

ells = np.arange(1, 8)
Cells = np.empty(len(ells))
for i, ell in enumerate(ells):
    # construct templates for fit
    ampss = np.identity(2 * ell + 1)
    templates = np.array([
        multipole_map(amps, NSIDE=NSIDE) for amps in ampss
    ])  # multipole_map() assumes ell from len(amps) and assumes that amps are given in order of increasing m
    
    # get the best-fit multipole moments
    pars, stderr = fit_multipole(map_to_fit, templates, idx=~np.isnan(map_to_fit))
    
    # compute Cell (exclude monopole)
    Cells[i] = compute_Cell(pars[1:])

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(ells, Cells, 'k.-')
# ax.plot(ells, ells * (ells + 1) * Cells, 'k.-')
ax.grid(lw=0.5, alpha=0.5)
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$\hat{C}_\ell$')
# ax.set_ylabel(r'$\ell\,(\ell +1)\,\hat{C}_\ell$')
ax.set_title(f'{d.catname}'r' low-$\ell$ power spectrum')